In [1]:
if 'google.colab' in str(get_ipython()):
    !pip install pytorch-lifestream

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 KB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 KB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 89.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 KB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 KB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 97.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 KB 18.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import pandas as pd
import torch

In [3]:
if not os.path.exists('data/train.csv'):
    ! mkdir -p data
    ! curl -OL https://storage.yandexcloud.net/datasouls-ods/materials/e756bf99/train.csv
    #! unzip -j -o transactions.zip '*.csv' -d data
    ! mv train.csv data/

if not os.path.exists('data/clickstream.csv'):
    ! mkdir -p data
    ! curl -OL https://storage.yandexcloud.net/datasouls-ods/materials/0554f0cf/clickstream.zip
    ! unzip -j -o clickstream.zip '*.csv' -d data
    ! mv clickstream.zip data/

if not os.path.exists('data/train_matching.csv'):
    ! mkdir -p data
    ! curl -OL https://storage.yandexcloud.net/datasouls-ods/materials/acfacf11/train_matching.csv
    # ! unzip -j -o clickstream.zip '*.csv' -d data
    ! mv train_matching.csv data/

print(f'Loaded csv files')

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  307k  100  307k    0     0   207k      0  0:00:01  0:00:01 --:--:--  207k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  469M  100  469M    0     0  14.8M      0  0:00:31  0:00:31 --:--:-- 16.1M
Archive:  clickstream.zip
  inflating: data/clickstream.csv    
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1045k  100 1045k    0     0   554k      0  0:00:01  0:00:01 --:--:--  554k
Loaded csv files


In [4]:
def trx_types(df):
    df['mcc_code'] = df['mcc_code'].astype(str)
    df['currency_rk'] = df['currency_rk'].astype(str)
    df['event_time'] = pd.to_datetime(df['transaction_dttm']).astype(int) / 1e9
    return df[['user_id', 'event_time', 'mcc_code', 'currency_rk', 'transaction_amt']]


def click_types(df):
    df['event_time'] = pd.to_datetime(df['timestamp']).astype(int) / 1e9
    # df = pd.merge(df, click_categories, on='cat_id')
    df['cat_id'] = df['cat_id'].astype(str)
    return df[['user_id', 'event_time', 'cat_id', 'new_uid']]


In [5]:
data_path = 'data/'

# Clickstream preprocessing

In [6]:
%%time
train_matching = pd.read_csv(os.path.join(data_path, 'train_matching.csv'))
train_matching.head()

CPU times: user 29.4 ms, sys: 10.3 ms, total: 39.7 ms
Wall time: 54 ms


,bank,rtk
0,178b387813ac4a63a2200274a9e3876e,e19c0f141e9442c5bdb0d5751f6fba1c
1,47cffa46e6b04389ba91a70735d2c3ff,7df3371aabd349e4ac2de8299238627d
2,f73b767cfd72472aa06df9a69e746aaf,b23d62c7e41145a7ad140a4fcfaafefa
3,48da649603734185b98dcea93484dbc7,63ad789541c54463ad54d0a707b3ab8b
4,37304ef19de542ee8fe8a6f050236525,c0e96de5dd594d948a8b9fde9e6980cb


In [8]:
target = pd.read_csv(os.path.join(data_path, 'train.csv'))
target.head()

,bank,higher_education
0,3755b59782464456bac1aec1a44e0db3,0.0
1,604a550439d644718ea6e1693fbf03dc,0.0
2,542d4776ebe5454fb8ab36f1c276fe0e,1.0
3,ee37fecea44d475ca030cde7ff7d545d,0.0
4,18617a1100f44a99b3a0772341fec3db,0.0


In [9]:
from tqdm import tqdm
pd.options.mode.chained_assignment = None  # to reduce warnings from pandas

clickstream_data = pd.read_csv(os.path.join(data_path, 'clickstream.csv'),
                          chunksize=100000)
clickstream = pd.DataFrame()

for chuck in tqdm(clickstream_data):
  #values, which id can be merged with target id via train_matching
  chuck = chuck[chuck['user_id'].isin(train_matching[train_matching['bank'].isin(target['bank'])]['rtk'])]
  chuck = click_types(chuck).dropna()
  clickstream = pd.concat([clickstream, chuck])

1268it [24:37,  1.17s/it]


In [10]:
clickstream = clickstream.reset_index(drop=True)
clickstream.head()

,user_id,event_time,cat_id,new_uid
0,000a8d3cdef3455d990e97730a2cef43,1.611327e+09,12,1364191
1,000a8d3cdef3455d990e97730a2cef43,1.611429e+09,931,531108
2,000a8d3cdef3455d990e97730a2cef43,1.611482e+09,165,531108
3,000a8d3cdef3455d990e97730a2cef43,1.611600e+09,503,531108
4,000a8d3cdef3455d990e97730a2cef43,1.611771e+09,931,531108


In [11]:
from ptls.preprocessing import PandasDataPreprocessor
preprocessor_cls = PandasDataPreprocessor(
    col_id='user_id',
    col_event_time='event_time',
    event_time_transformation='none',
    cols_category=["cat_id", "new_uid"],
    cols_numerical=[],
    return_records=True,
)

In [ ]:
%%time
dataset_clickstream = preprocessor_cls.fit_transform(clickstream)
print(f'Clickstream features prepared')

In [ ]:
import pickle
with open('dataset_clickstream', 'wb') as ds:
    pickle.dump(dataset_clickstream, ds)